In [1]:
# Web Pages
# MLB - https://www.baseball-reference.com/leagues/MLB-schedule.shtml#today
# NFL - https://www.pro-football-reference.com/years/2023/games.htm
# NBA - https://www.basketball-reference.com/leagues/NBA_2024_games.html
# NHL - https://www.hockey-reference.com/leagues/NHL_2024_games.html
# MLS - https://fbref.com/en/comps/22/schedule/Major-League-Soccer-Scores-and-Fixtures

In [14]:
!pip install webdriver-manager

In [16]:
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [29]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get("https://www.baseball-reference.com/leagues/MLB-schedule.shtml")
dr

In [36]:
driver.close()

# MLB

In [43]:
# Read in MLB schedule data with BeautifulSoup
browser = Browser('chrome', service=ChromeService(ChromeDriverManager().install()))
browser.visit('https://www.baseball-reference.com/leagues/MLB-schedule.shtml#today')

mlb_html = browser.html
mlb_soup = BeautifulSoup(mlb_html, 'html.parser')

In [58]:
full_schedule = mlb_soup.find('div', id='div_164218044')

date_divs = full_schedule.find_all('div')

date_divs[151]

<div>
<h3><span id="today">Today's Games</span></h3>
<p class="game">
<span tz="E"><strong>1:10 pm</strong></span>
<a href="/teams/CHC/2023.shtml">Chicago Cubs</a>
 
 @
  <a href="/teams/CIN/2023.shtml">Cincinnati Reds</a>
 
     <em><a href="/previews/2023/CIN202309011.shtml">Preview</a></em>
</p>
<p class="game">
<span tz="E"><strong>6:40 pm</strong></span>
<a href="/teams/CHC/2023.shtml">Chicago Cubs</a>
 
 @
  <a href="/teams/CIN/2023.shtml">Cincinnati Reds</a>
 
     <em><a href="/previews/2023/CIN202309012.shtml">Preview</a></em>
</p>
<p class="game">
<span tz="E"><strong>7:05 pm</strong></span>
<a href="/teams/MIA/2023.shtml">Miami Marlins</a>
 
 @
  <a href="/teams/WSN/2023.shtml">Washington Nationals</a>
 
     <em><a href="/previews/2023/WAS202309010.shtml">Preview</a></em>
</p>
<p class="game">
<span tz="E"><strong>7:10 pm</strong></span>
<a href="/teams/TBR/2023.shtml">Tampa Bay Rays</a>
 
 @
  <a href="/teams/CLE/2023.shtml">Cleveland Guardians</a>
 
     <em><a href="/pre

In [59]:
passed_today = 0
mlb_schedule_dict = {
    'date':[],
    'time':[],
    'away':[],
    'home':[]
}

for div in date_divs:
    date = div.find('h3')
    if date.find('span'):
        passed_today = 1
    if passed_today == 1:
        games = div.find_all(class_='game')
        for game in games:
            mlb_schedule_dict['date'].append(date.text)
            time = game.find('span').text
            mlb_schedule_dict['time'].append(time)
            teams = game.find_all('a')
            mlb_schedule_dict['away'].append(teams[0].text)
            mlb_schedule_dict['home'].append(teams[1].text)
            
            
        


In [60]:
pd.DataFrame(mlb_schedule_dict

{'date': ["Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  "Today's Games",
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Saturday, September 2, 2023',
  'Sunday, September 3, 2023',
  'Sunday, September 3, 2023',
  'Sunday, September 3, 2023',
  'Sunday, September 3, 2023',
  'Sunday, September 3, 2023',
  'Sunday, September 3, 2023',
  'Sund